In [ ]:
import numpy as np

import pandas as pd
from pandas.tools.plotting import scatter_matrix

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

from scipy import optimize

from sklearn.preprocessing import PolynomialFeatures

import csv

import time

# 1 Univariate Logistic Regression

In [ ]:
dataFile_1 = 'ex2data1.txt'

# 1.1 Data Visualisation

In [ ]:
dF_1 = pd.read_csv(dataFile_1, header = None, names = ['x1', 'x2', 'y'])
sampleSize_1, nVariables_1 = dF_1.shape
print(dF_1.head())
print ("sampleSize =", sampleSize_1, "nVariables =", nVariables_1)

In [ ]:
x1 = dF_1['x1']
x2 = dF_1['x2']
plt.figure(figsize=(8,6))
plt.scatter(x1, x2, c = dF_1['y'])
plt.xlabel("X1")
plt.ylabel("X2")
plt.show()

## 1.2 Data Extraction and Transformation

In [ ]:
def getData(dataFile):
#     try wiht matrices as well
    data = np.loadtxt(dataFile, delimiter = ',')
    sampleSize, nVariables = data.shape
    X = np.insert(data[:, :-1], 0, 1, axis=1)
    y = data[:, -1:]
#     beta = np.matrix(np.zeros(nVariables)).T
    beta = np.zeros(nVariables)
    return beta, X.flatten(), y.flatten(), sampleSize, nVariables

## 1.3 Logistic Regression
### 1.3.1 Logistic Regression

**Sigmoid Function** ${\sigma}(z) = \frac{1}{1 + e^{-z}}$


### 1.3.2 Vectorisation of Logistic Regression

**Hypothesis** $h_{\beta}(X) =   \frac{1}{1 + e^{X\cdot\beta}}$

**Cost Function** $J = \frac{-1}{n}\sum(y^T\cdot \log h_{\beta} +(1-y)^T\cdot \log (1-h_{\beta}))$

In [ ]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

def hypothesis(beta, X, sampleSize, nVariables):
    beta = beta.reshape(nVariables, -1)
    X = X.reshape(sampleSize, -1)
    return sigmoid(np.dot(X, beta))

# def costFunction(X, y, beta):
def costFunction(beta, X, y, sampleSize, nVariables, iLambda = 0.):
#     beta = beta.reshape(nVariables, -1)
#     X = X.reshape(sampleSize, -1)
    y = y.reshape(sampleSize, -1)
#     hypothesis vector h(n, 1)
    h = hypothesis(beta, X, sampleSize, nVariables)
#     cost scalar J(1, 1)
    J = (- np.dot(y.T, np.log(h)) - np.dot((1-y).T, np.log(1-h)))/sampleSize
#     similarly cost J can be calculated using np.multiply together with np.sum
#     cost = -np.sum(np.multiply(y, np.log(h)) + np.multiply((1-y), np.log(1-h)))/sampleSize
#     regularisation scalar (R)
    R = iLambda*np.dot(beta[1:].T,beta[1:])/(2*sampleSize)
    return (J + R)[0][0]

def betaOptimisation_1 (beta, X, y, sampleSize, nVariables, iLambda=0.):
    return optimize.fmin(costFunction, x0=beta, args=(X, y, sampleSize, nVariables, iLambda), maxiter=1500, full_output=True)

def prediction(beta, X, sampleSize, nVariables):
    return hypothesis(beta, X, sampleSize, nVariables) >= 0.5

## 1.4 Function Tests

In [ ]:
betaTest_1, X_1, y_1, sampleSize_1, nVariables_1 = getData(dataFile_1)
y_1.shape

### 1.4.1 Cost-Function Test
The outputs of the costFunction should be as follows:<br\>
betaTest (set to zeros), X, iLambda=0. — **J = 0.693** (Andrew Ng) <br\>

In [ ]:
print("J =", costFunction(betaTest_1, X_1, y_1, sampleSize_1, nVariables_1))

### 1.5.1 Prediction Test
The outputs of the costFunction should be as follows:<br\>
Exam_1: 45, Exam_2: 85 — **P = 0.776** (Andrew Ng) <br\>

In [ ]:
betaOpt_1 = betaOptimisation_1(betaTest_1, X_1, y_1, sampleSize_1, nVariables_1)[0]
xTest_1 = np.array([1, 45, 85])
sampleSizeTest_1 = 1
print("P =", hypothesis(betaOpt_1, xTest_1, sampleSizeTest_1, nVariables_1)[0][0])

## 1.5 Results Visualisation & Analysis
### 1.5.1 Goodness of Fit Measures
#### 1.5.1.1 Decision Boundary
This comment is here thanks to this dude (https://github.com/vsevolodloik).<br />
Decision boundary is defined as follows:<br />
$\frac{1}{1 + e^{X\cdot\beta}} = \frac{1}{2}$<br />
Therefore, for the simple case of two variables, the equation of decision boundary takes the following form:<br />
$\beta_0+\beta_1\cdot{X_1}+\beta_2 \cdot{X_2} = 0$
#### 1.5.1.2 Types of Errors & Accuracy, Precision, Recal

The rate **type I error** (false positives) is denoted by $\alpha$.<br />
The rate **type II error** (false negatives) is denoted by $\beta$.<br /><br />
**Accuracy** $= \frac {tP + tN}{tP + tN + fP + fN}$<br /><br />
**Precision** $= \frac {tP}{tP + fP}$<br /><br />
**Recall** $= \frac {tP}{tP + fN}$

In [ ]:
def goodnessOfFit(beta, X, y,  sampleSize, nVariables):
    beta_R = beta.reshape(nVariables, -1)
    X_R = X.reshape(sampleSize, -1)
    y_R = y.reshape(sampleSize, -1)
    p = prediction(beta, X, sampleSize, nVariables).flatten()
    
#     Elegant way to calculate tP, fP, and fN
    tP = np.sum(y*p)
    fP = np.sum(y-p==-1)
    fN = np.sum(y-p==1)
    precision  = tP/(tP+fP)
    recall  = tP/(tP+fN)
    accuracy = (X.shape[0] - fP - fN)/X.shape[0]
    print("Accuracy", accuracy, "\nPrecision =", precision, "\nRecall =", recall)
    
    plt.figure(figsize=(8,6))
    x1 = X_R[:, 1:2]
    x2 = X_R[:, 2:]
    plt.scatter(x1, x2, c = y_R[:, 0:])
    x2Fit = - beta_R[0]/beta_R[2] - x1*beta_R[1]/beta_R[2]
    plt.plot(x1, x2Fit, '-')
    plt.xlabel("X1")
    plt.ylabel("X2")
    return plt.show()

In [ ]:
goodnessOfFit(betaOpt_1, X_1, y_1, sampleSize_1, nVariables_1)

http://www.johnwittenauer.net/tag/machine-learning/

http://aimotion.blogspot.se/2011/11/machine-learning-with-python-logistic.html

https://beckernick.github.io/logistic-regression-from-scratch/

https://github.com/kaleko/CourseraML/blob/master/ex2/ex2.ipynb

http://www.scipy-lectures.org/advanced/mathematical_optimization/

# 2 Multivariate Logistic Regression

In [ ]:
dataFile_2 = 'ex2data2.txt'
dF_2 = pd.read_csv(dataFile_2, header = None)
sampleSize, nVariables = dF_2.shape
print ("sampleSize =", sampleSize, "nVariables =", nVariables)
print (dF_2.head())

## 2.1 Data Visualisation

In [ ]:
X_1s = dF_2.iloc[:, :1]
X_2s = dF_2.iloc[:, 1:2]
plt.figure(figsize=(8,6))
plt.scatter(X_1s, X_2s, c = dF_2.iloc[:, 2:])
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()

## 2.2 Data Extraction Transformation
Add **polynomial** and **interaction** features using **SciKitLearn Preprocessing**<br\>
http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

In [ ]:
def addPolynomial(dataFile, polynomialDegree):
    data = np.loadtxt(dataFile, delimiter = ',')
    sampleSize, nVariables = data.shape
    X = data[:, :-1]
    y = data[:, -1:]
    poly = PolynomialFeatures(polynomialDegree)
#     X without intercept is passed to PolynomialFeatures.fit_transform.
#     Intercept is added automatically.
    polyX = poly.fit_transform(X)
    sampleSize, nVariables = polyX.shape
    beta = np.zeros((nVariables,1))
    return beta.flatten(), polyX.flatten(), y.flatten(), sampleSize, nVariables

## 2.3 Function Tests

In [ ]:
betaPoly6, XPoly6, yPoly6, sampleSizePoly6, nVariablesPoly6 = addPolynomial(dataFile_2, 6)

### 2.3.1 Cost-Function Test
The outputs of the costFunction should be as follows:<br\>
betaTest (set to zeros), X, iLambda=0. — **J = 0.693** (Andrew Ng) <br\>

In [ ]:
print("J =",costFunction(betaPoly6, XPoly6, yPoly6, sampleSizePoly6, nVariablesPoly6))

In [ ]:
def betaOptimisation_2(beta, X, y, sampleSize, nVariables, iLambda=0.):

    optimisedBeta = optimize.minimize(costFunction, beta, args=(X, y, sampleSize, nVariables, iLambda),
                                      method='BFGS', options={'maxiter':50})

#     optimisedBeta = optimize.fmin_cg(costFunction, fprime=backPropagation, x0=flatBeta,
#                                      args=(layer, flatX, sampleSize, y, yUnique),
#                                      maxiter=50,disp=True,full_output=True)
    return(optimisedBeta['x'])

In [ ]:
# betaOpt = betaOptimisation(betaPoly6, XPoly6, yPoly6, iLambda = 0.)[0]

## 2.4 Results Visualisation & Analysis

In [ ]:
def decisionBoundary(beta, X, y, sampleSize, nVariables, xMin, xMax, step, polyOrder, iLambda=0.):
    p = prediction(beta, X, sampleSize, nVariables).flatten()
    tP = np.sum(y*p)
    fP = np.sum(y-p==-1)
    fN = np.sum(y-p==1)
    precision  = tP/(tP+fP)
    recall  = tP/(tP+fN)
    accuracy = (X.shape[0] - fP - fN)/X.shape[0]
    print("Accuracy", accuracy, "\nPrecision =", precision, "\nRecall =", recall)
    
    x1 = np.linspace(xMin[0], xMax[0], step)
    x2 = np.linspace(xMin[1], xMax[1], step)
    X1, X2 = np.meshgrid(x1, x2)
    combinedX = np.concatenate((X1.reshape(step**2, -1), X2.reshape(step**2, -1)), axis=1)
    # X without intercept is passed to PolynomialFeatures.fit_transform.
    # Intercept is added automatically.
    poly = PolynomialFeatures(polyOrder)
    polyX = poly.fit_transform(combinedX)
    Y = hypothesis(beta, polyX, step**2, polyX.shape[1])
    # Y = prediction(betaOpt_2, polyX, 2500, nVariablesPoly6)
    Y.reshape(step, -1)
    
#     plt.figure(figsize=(8,6))
    decisionBoundary = plt.contour(X1, X2, Y.reshape(step, -1), [0.5])
#     label = {0:'Lambda = %d'%iLambda}
    plt.clabel(decisionBoundary, inline=1, fontsize=10, fmt = '$\lambda $= %d'%iLambda)

    plt.title("Decision Boundary")

    x1s = X.reshape(sampleSize,-1)[:, 1:2]
    x2s = X.reshape(sampleSize,-1)[:, 2:3]
    plt.scatter(x1s, x2s, c = y.reshape(sampleSize,-1)[:, 0:])

    plt.xlabel("X1")
    plt.ylabel("X2")
    return plt.show()

In [ ]:
iLambda = 0
polyOrder = 6
betaPoly, XPoly, yPoly, sS_Poly, nV_Poly = addPolynomial(dataFile_2, polyOrder)
betaOpt_2 = betaOptimisation_2(betaPoly, XPoly, yPoly, sS_Poly, nV_Poly, iLambda)

xMin = (-1., -1.)
xMax = (1.2, 1.2)
step = 50
decisionBoundary(betaOpt_2, XPoly, yPoly, sS_Poly, nV_Poly, xMin, xMax, step, polyOrder, iLambda)

In [ ]:
for i, iLambda in enumerate([0., 1., 10, 100 ]):
    polyOrder = 6
    betaPoly, XPoly, yPoly, sampleSizePoly, nVariablesPoly = addPolynomial(dataFile_2, polyOrder)
    betaOpt = betaOptimisation_2(betaPoly, XPoly, yPoly, sampleSizePoly, nVariablesPoly, iLambda)
    xMin = (-1., -1.)
    xMax = (1.2, 1.2)
    step = 50
    decisionBoundary(betaOpt, XPoly, yPoly, sampleSizePoly, nVariablesPoly, xMin, xMax, step, polyOrder, iLambda)